[Reference](https://medium.com/@kasperjuunge/how-to-use-fastapi-users-cae8ed1058f8)

In [11]:
!pip install fastapi fastapi-users sqlmodel sqlite

In [12]:
from sqlmodel import SQLModel, Field
from fastapi_users.db import SQLModelBaseUser

class User(SQLModelBaseUser, table=True):
    first_name: str = Field(nullable=False)
    last_name: str = Field(nullable=False)

In [13]:
from fastapi_users import models

class UserCreate(models.CreateUpdateDictModel):
    first_name: str
    last_name: str
    email: str
    password: str

In [14]:
class UserUpdate(models.CreateUpdateDictModel):
    first_name: Optional[str] = None
    last_name: Optional[str] = None

In [15]:
class UserDB(User, models.BaseUserDB):
    pass

In [9]:
from sqlmodel import SQLModel, create_engine

sqlite_file_name = "database.db"
sqlite_url = f"sqlite:///{sqlite_file_name}"
engine = create_engine(sqlite_url)

def create_db_and_tables():
    SQLModel.metadata.create_all(engine)

In [8]:
from fastapi import FastAPI
from fastapi_users import FastAPIUsers
from fastapi_users.db import SQLModelUserDatabase

app = FastAPI()

# Setup User manager
user_db = SQLModelUserDatabase(UserDB, engine)
fastapi_users = FastAPIUsers(user_db, [jwt_authentication], User, UserCreate, UserUpdate, UserDB)

# User routes
app.include_router(
    fastapi_users.get_auth_router(jwt_authentication), prefix="/auth/jwt", tags=["auth"]
)
app.include_router(
    fastapi_users.get_register_router(UserCreate), prefix="/auth", tags=["auth"]
)
app.include_router(
    fastapi_users.get_users_router(User, UserCreate, UserUpdate), prefix="/users", tags=["users"]
)

@app.on_event("startup")
def on_startup():
    create_db_and_tables()